# ASSIGNMENT 6

## Preparation

### 2. Setup/import

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from keras.utils import load_img, img_to_array

from sklearn.metrics import accuracy_score

import glob
from google.colab import drive
import os
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## 1. Data Processing: 
The train & test data is pretty clean in terms of image data, but we will need to do a bit of prep work to use in our model.

a) Use the "ImageDataGenerator()" class from keras.processing.image to build out an instance called "train_datagen" with the following parameters: 
- rescale = 1./255
- shear_range = 0.2
- zoom_range = 0.2
- horizontal_flip = True

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True)

b) Then build your training set by using the method ".flow_from_directory()"
- path (where training data is stored)
- target_size = (64, 64)
- batch_size = 32
- class_mode = categorical 

In [ ]:
directory = 'gdrive/My Drive/dataset_train'
train_set = train_datagen.flow_from_directory(directory,
                                  target_size = (64, 64),
                                  batch_size = 32,
                                  class_mode = 'categorical')

Found 88 images belonging to 4 classes.


c) Take a look at your training set: 

- What is the image shape of each training observation?
- How many total classes do we need to predict on?

In [ ]:
print("Image shape:", train_set.image_shape)
print("Number of classes:", train_set.num_classes)

Image shape: (64, 64, 3)
Number of classes: 4


# 2. Initial Classifier Build: 
Now use keras to build an initial image classifier with the following specifications.


- Create an instance of Sequential called "classifier"

In [ ]:
classifier = Sequential()

- Add a Conv2D layer with the following parameters: 
    - filters = 32
    - kernel_size = (3,3)
    - input_shape = image shape found in part 1
    - activation = relu

In [ ]:
classifier.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=train_set.image_shape, activation='relu'))

- Add a MaxPooling2D layer where pool_size = (2,2)

In [ ]:
classifier.add(MaxPooling2D(pool_size=(2, 2)))

- Add another Conv2D layer: 
    - filters = 64
    - kernel_size = (3,3)
    - activation = relu

In [ ]:
classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

- Add a MaxPooling2D layer where pool_size = (2,2)

In [ ]:
classifier.add(MaxPooling2D(pool_size=(2, 2)))

- Add a Flatten layer

In [ ]:
classifier.add(keras.layers.Flatten())

- Add a Dense layer
    - units = 128
    - activation = relu

In [ ]:
classifier.add(keras.layers.Dense(units = 128, activation = 'relu'))

- Add a final Dense layer (this will output our probabilities):
    - units = # of classes
    - activation = softmax 

In [ ]:
classifier.add(Dense(units = train_set.num_classes, activation = 'softmax'))

- Compile with the following: 
    - optimize = adam
    - loss = categorical cross entropy
    - metric = accuracy

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## 3. Model Runs: 

This will be run various times with different numbers of steps_per_epoch and epochs.

a) Use .fit() with the training set. For the first run, use the following parameters:
- steps_per_epoch = 3
- epochs = 3

In [ ]:
classifier.fit(train_set, steps_per_epoch=3, epochs=3)

Epoch 1/3
3/3 [==============================] - 4s 212ms/step - loss: 1.9468 - accuracy: 0.3409
Epoch 2/3
3/3 [==============================] - 1s 319ms/step - loss: 1.1827 - accuracy: 0.5341
Epoch 3/3
3/3 [==============================] - 1s 325ms/step - loss: 0.5965 - accuracy: 0.8864


b) save model to a file. An example is below:
```
# save model
classifier.save('my_model.h5')
print("Saved model")
```

In [ ]:
classifier.save('my_model.h5')
print("Saved model")

Saved model


c) Predict using the model built in step 2.

In [ ]:
# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')
print("Loaded model from disk")

# test data path
img_dir = "gdrive/My Drive/dataset_test"

data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)

# print the files in the dataset_test folder 
for f in files:
    print(f)

# make a prediction and add to results 
data = []
results = []
for f1 in files:
    img = load_img(f1, target_size = (64, 64))
    img = img_to_array(img)           # Convert the image to an array
    img = np.expand_dims(img, axis = 0)     # Add a fourth dimension to the image array to match input shape of the model
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results

Loaded model from disk
gdrive/My Drive/dataset_test/6051.png
gdrive/My Drive/dataset_test/4053.png
gdrive/My Drive/dataset_test/1053.png
gdrive/My Drive/dataset_test/1022.png
gdrive/My Drive/dataset_test/C033.png
gdrive/My Drive/dataset_test/C014.png
gdrive/My Drive/dataset_test/6023.png
gdrive/My Drive/dataset_test/4011.png
1/1 [==============================] - 0s 27ms/step


[array([1]),
 array([2]),
 array([0]),
 array([0]),
 array([3]),
 array([1]),
 array([1]),
 array([1])]

d) Determine accuracy.

In [ ]:
# check category labels in training_set
train_set.class_indices

{'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}

In [ ]:
test_label = [1, 2, 0, 0, 3, 3, 1, 2]

# calculate accuracy score
accuracy = accuracy_score(test_label, results)
print("Accuracy:", accuracy)

Accuracy: 0.75


e) Run this process for the following combinations:

* (steps_per_epoch: 1, epochs: 1)
* (steps_per_epoch: 1, epochs: 2)
* (steps_per_epoch: 1, epochs: 3)
* (steps_per_epoch: 2, epochs: 4)
* (steps_per_epoch: 2, epochs: 5)
* (steps_per_epoch: 2, epochs: 6)
* (steps_per_epoch: 3, epochs: 7)
* (steps_per_epoch: 3, epochs: 8)
* (steps_per_epoch: 5, epochs: 9)
* (steps_per_epoch: 5, epochs: 10)

In [ ]:
# def determine_acc(s, e, steps_list, epochs_list, accuracy_list, train_accuracy_list, history_list):
#     classifier = Sequential()
#     classifier.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=train_set.image_shape, activation='relu'))
#     classifier.add(MaxPooling2D(pool_size=(2, 2)))
#     classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
#     classifier.add(MaxPooling2D(pool_size=(2, 2)))
#     classifier.add(keras.layers.Flatten())
#     classifier.add(keras.layers.Dense(units = 128, activation = 'relu'))
#     classifier.add(Dense(units = train_set.num_classes, activation = 'softmax'))
#     classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

#     history = classifier.fit(train_set, steps_per_epoch = s, epochs = e)
#     model_name = 'Model_' + str(s) + '_' + str(e) + '.h5'
#     classifier.save(model_name)
#     print(train_set.class_indices)
#     print("Saved model: " + model_name)

#     model = load_model(model_name)
#     print("Loaded model from disk: " + model_name)

#     # make a prediction and add to results 
#     data = []
#     results = []
#     for f1 in files:
#         img = load_img(f1, target_size = (64, 64))
#         img = img_to_array(img)           # Convert the image to an array
#         img = np.expand_dims(img, axis = 0)     # Add a fourth dimension to the image array to match input shape of the model
#         data.append(img)
#         result = model.predict(img)
#         r = np.argmax(result, axis=1)
#         results.append(r)
    
#     print("The Loss = " + str(history.history['loss']))
#     accuracy = accuracy_score(test_label, results)
#     steps_list.append(s)
#     epochs_list.append(e)
#     accuracy_list.append(accuracy)
#     history_list.append(history.history['loss'][len(history.history['loss'])-1])
#     train_accuracy_list.append(history.history['accuracy'][len(history.history['accuracy'])-1])

#     return steps_list, epochs_list, accuracy_list, train_accuracy_list, history_list

# combinations = [(1, 1), (1,2), (1, 3), (2, 4), (2, 5), (2,6), (3, 7), (3, 8), (5, 9), (5, 10)]

# steps_list = []
# epochs_list = []
# accuracy_list = []
# history_list =[]
# train_accuracy_list = []

# for s, e in combinations:
#     steps_list, epochs_list, accuracy_list, train_accuracy_list, history_list = determine_acc(s, e, steps_list, epochs_list, accuracy_list, train_accuracy_list, history_list)
#     print()
#     print()



In [ ]:
n_batch = 5
n_epochs = 5

combinations = [(1, 1), (1, 2), (1, 3), (2, 4), (2, 5), (2, 6), (3, 7), (3, 8), (5, 9), (5, 10)]

steps_list = []
epochs_list = []
accuracy_list = []
history_list =[]
train_accuracy_list = []

for s, e in combinations:
    print('Steps:', s, 'Epochs:', e)
    print('--------------')
    
    # fit the model using generator
    for epoch in range(e):
        print('Epoch', epoch)
        batches = 0
        for x_batch, y_batch in train_datagen.flow_from_directory(directory, target_size=(64, 64), batch_size = 32, class_mode = 'categorical', seed = 74):
            classifier = Sequential()
            classifier.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=train_set.image_shape, activation='relu'))
            classifier.add(MaxPooling2D(pool_size=(2, 2)))
            classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
            classifier.add(MaxPooling2D(pool_size=(2, 2)))
            classifier.add(keras.layers.Flatten())
            classifier.add(keras.layers.Dense(units = 128, activation = 'relu'))
            classifier.add(Dense(units = train_set.num_classes, activation = 'softmax'))
            classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
            history = classifier.fit(x_batch, y_batch, steps_per_epoch = s, epochs = e)
            batches += 1
            if batches >= n_batch:
                break

    # save model and update lists
    model_name = 'Model_' + str(s) + '_' + str(e) + '.h5'
    classifier.save(model_name)
    print('Saved model:', model_name)

    model = load_model(model_name)
    print("Loaded model from disk: " + model_name)

    # determine accuracy using test set
    data = []
    results = []
    for f1 in files:
        img = load_img(f1, target_size=(64, 64))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        data.append(img)
        result = model.predict(img)
        r = np.argmax(result, axis=1)
        results.append(r)

    print("The Loss = " + str(history.history['loss']))
    accuracy = accuracy_score(test_label, results)
    steps_list.append(s)
    epochs_list.append(e)
    accuracy_list.append(accuracy)
    history_list.append(history.history['loss'][len(history.history['loss'])-1])
    train_accuracy_list.append(history.history['accuracy'][len(history.history['accuracy'])-1])

    print('Accuracy:', accuracy)
    print('--------------')

    print()
    print()
    
# create a dataframe from the lists
df = pd.DataFrame({
    'Steps': steps_list,
    'Epochs': epochs_list,
    'Accuracy': accuracy_list,
    'Train Loss': history_list,
    'Train Accuracy': train_accuracy_list
})


Steps: 1 Epochs: 1
--------------
Epoch 0
Found 88 images belonging to 4 classes.
1/1 [==============================] - 2s 2s/step - loss: 1.4008 - accuracy: 0.2500


1/1 [==============================] - 1s 1s/step - loss: 1.4318 - accuracy: 0.1562


1/1 [==============================] - 1s 1s/step - loss: 1.4270 - accuracy: 0.0312
Saved model: Model_1_1.h5
Loaded model from disk: Model_1_1.h5
1/1 [==============================] - 0s 19ms/step
The Loss = [1.4270329475402832]
Accuracy: 0.25
--------------


Steps: 1 Epochs: 2
--------------
Epoch 0
Found 88 images belonging to 4 classes.
Epoch 1/2
1/1 [==============================] - 1s 1s/step - loss: 1.4295 - accuracy: 0.0312
Epoch 2/2
1/1 [==============================] - 0s 12ms/step - loss: 1.1191 - accuracy: 0.5625
Epoch 1/2
1/1 [==============================] - 1s 1s/step - loss: 1.3731 - accuracy: 0.2812
Epoch 2/2
1/1 [==============================] - 0s 12ms/step - loss: 1.1274 - accuracy: 0.5000
Epoch 1/2
1/1 [==============================] - 1s 1s/step - loss: 1.4115 - accuracy: 0.2917
Epoch 2/2
1/1 [==============================] - 0s 11ms/step - loss: 2.3996 - accuracy: 0.3750
Epoch 1/2
1/1 [==============================] - 1s 1s/step - loss: 1.3985 - accuracy

f) Create a final dataframe that combines the accuracy across each combination.

In [ ]:
df[['Steps', 'Epochs', 'Accuracy']].style.hide(axis='index')

Steps,Epochs,Accuracy
1,1,0.250000
1,2,0.250000
1,3,0.625000
2,4,0.500000
2,5,0.875000
2,6,1.000000
3,7,0.750000
3,8,0.750000
5,9,0.875000
5,10,0.750000


In [ ]:
df.style.hide(axis='index')

Steps,Epochs,Accuracy,Train Loss,Train Accuracy
1,1,0.250000,1.427033,0.031250
1,2,0.250000,2.099864,0.250000
1,3,0.625000,0.968235,0.531250
2,4,0.500000,0.801228,0.781250
2,5,0.875000,0.375988,0.906250
2,6,1.000000,0.604526,0.875000
3,7,0.750000,0.020536,1.000000
3,8,0.750000,0.049375,1.000000
5,9,0.875000,0.001152,1.000000
5,10,0.750000,0.002477,1.000000


## Conceptual Questions: 

4. Discuss the effect of the following on accuracy and loss (train & test): 
- Increasing the steps_per_epoch
- Increasing the number of epochs



> We can see that increasing both the steps_per_epoch and epochs generally improves the training accuracy and reduces the training loss. This indicates that the model is learning more from the data as it is being exposed to it for longer periods of time. However, the effect on the test accuracy and loss is more mixed, with some increases and decreases observed.

> Specifically, we can observe that the training accuracy consistently improves as the number of steps_per_epoch and epochs increase. Similarly, the training loss consistently reduces as the number of steps_per_epoch and epochs increase.

> On the other hand, the effect on the test accuracy and loss is more varied. For example, increasing from (1,1) to (1,3) improves the test accuracy, but increasing from (1,3) to (2,4) actually reduces the test accuracy. Similarly, increasing from (1,2) to (2,6) dramatically improves the test accuracy, but increasing from (3,7) to (5,9) has a smaller effect. This suggests that simply increasing the steps_per_epoch and epochs may not always lead to better performance on the test set, and that the optimal values may depend on the specific dataset and model being used.



5. Name two uses of zero padding in CNN.
Zero padding is a technique used in convolutional neural networks (CNNs) that involves adding zeros around the borders of an input image or feature map. This technique can be used for two main purposes. 
- Firstly, it can help to preserve the spatial dimensions of an input image or feature map as it is passed through convolutional layers, which can be useful for tasks such as object detection and segmentation. 
- Secondly, it can help to prevent the loss of information at the borders of an input image or feature map during convolution, which can be important for maintaining the overall quality and accuracy of a CNN's output.

6. What is the use of a 1 x 1 kernel in CNN? <br>
The use of a 1 x 1 kernel in CNNs is to perform dimensionality reduction and compression on the feature maps. This kernel acts like a filter and is applied to each pixel of the input image, allowing the network to combine information from different channels at each location. By reducing the number of channels in a feature map, a 1 x 1 convolution can help to reduce the computational cost of the network while also improving its performance by making it more robust to overfitting. Additionally, a 1 x 1 convolution can be used as a bottleneck layer, where it is placed between layers with a high number of filters to reduce the number of features and computational cost without losing too much information.

7. What are the advantages of a CNN over a fully connected DNN for this image classification problem?
- Parameter sharing: In CNNs, the same filters are used across the entire image, which means that the number of parameters is significantly reduced compared to a fully connected DNN. This makes CNNs more efficient to train and requires less memory.

- Translation invariance: CNNs can detect features regardless of their position in the image. This is achieved by using filters that slide across the entire image, which enables CNNs to capture the spatial dependencies between pixels.

- Local receptive fields: CNNs use local receptive fields, which allow them to capture the local structure of the image. This means that the filters only operate on a small region of the image at a time, which helps to preserve the spatial structure of the image.

- Hierarchical representation: CNNs are able to learn hierarchical representations of the image, where low-level features are combined to form higher-level features. This enables CNNs to capture complex patterns in the image and leads to improved classification accuracy.

> Overall, CNNs are a powerful tool for image classification due to their ability to capture spatial dependencies and learn hierarchical representations of the image.